In [1]:
!pip install torchmetrics
!pip install thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 17.4 MB/s eta 0:00:00


In [2]:
#import

import zipfile
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from collections import OrderedDict
from PIL import Image
import matplotlib.pyplot as plt
from thop import profile, clever_format

import torch.nn as nn
import torch.optim as optim

from torchmetrics import JaccardIndex
import torch.nn.functional as F
import random

from tqdm import tqdm

import time
import logging



In [3]:
!wget https://zenodo.org/record/5706578/files/Train.zip
!wget https://zenodo.org/record/5706578/files/Val.zip

--2025-01-09 21:37:40--  https://zenodo.org/record/5706578/files/Train.zip
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/5706578/files/Train.zip [following]
--2025-01-09 21:37:40--  https://zenodo.org/records/5706578/files/Train.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 4021669263 (3.7G) [application/octet-stream]
Saving to: ‘Train.zip’

Train.zip           100%[===================>]   3.75G  13.0MB/s    in 4m 58s  

2025-01-09 21:42:39 (12.9 MB/s) - ‘Train.zip’ saved [4021669263/4021669263]

--2025-01-09 21:42:39--  https://zenodo.org/record/5706578/files/Val.zip
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request 

In [4]:
!wget https://zenodo.org/records/14606189/files/PIDNet_S_ImageNet.pth.tar

--2025-01-09 21:44:41--  https://zenodo.org/records/14606189/files/PIDNet_S_ImageNet.pth.tar
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38061375 (36M) [application/octet-stream]
Saving to: ‘PIDNet_S_ImageNet.pth.tar’

PIDNet_S_ImageNet.p 100%[===================>]  36.30M  12.5MB/s    in 2.9s    

2025-01-09 21:44:45 (12.5 MB/s) - ‘PIDNet_S_ImageNet.pth.tar’ saved [38061375/38061375]



In [5]:
# Define file paths
train_zip = "Train.zip"
val_zip = "Val.zip"

# Extract Train.zip
with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall("Train")

# Extract Val.zip
with zipfile.ZipFile(val_zip, 'r') as zip_ref:
    zip_ref.extractall("Val")


# Verify the extracted folders
print("Train Directory Contents:", os.listdir("Train"))
print("Val Directory Contents:", os.listdir("Val"))

Train Directory Contents: ['Train']
Val Directory Contents: ['Val']


In [6]:
#TASK 3.b

COLOR_MAP = {
    'Background': 0.00392157,
    'Building': 0.00784314,
    'Road': 0.01176471,
    'Water': 0.01568628,
    'Barren': 0.01960784,
    'Forest': 0.02352941,
    'Agricultural': 0.02745098
}

LABEL_MAP = OrderedDict(
    Background=0,
    Building=1,
    Road=2,
    Water=3,
    Barren=4,
    Forest=5,
    Agricultural=6
)

# Tolerance-based color to label conversion
def color_to_one_hot(mask, label_map, tolerance=0.001, num_classes=7):
    mask = np.asarray(mask, dtype=np.float32)
    if len(mask.shape) == 2:  # Ensure channel dimension exists
        mask = np.expand_dims(mask, axis=0)  # Convert [H, W] to [1, H, W]

    one_hot_mask = np.zeros((num_classes, mask.shape[1], mask.shape[2]), dtype=np.float32)

    for class_name, class_index in label_map.items():
        color_value = COLOR_MAP[class_name]
        # Match pixels with the grayscale value within the tolerance
        matches = np.abs(mask - color_value) < tolerance
        one_hot_mask[class_index, np.squeeze(matches, axis=0)] = 1.0

    return torch.from_numpy(one_hot_mask)


# Define a function to denormalize the image
def denormalize(tensor, mean, std):
    """ Denormalize the tensor back to the [0, 1] range for visualization. """
    for i in range(len(mean)):
        tensor[i] = tensor[i] * std[i] + mean[i]
    return tensor

def show_image(image_tensor):
    """ Display a transformed image using matplotlib. """
    image_tensor = denormalize(image_tensor, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Denormalize the image
    image = image_tensor.permute(1, 2, 0).numpy()  # Convert to HxWxC format
    plt.imshow(image)
    plt.axis('off')  # Hide axes
    plt.show()

def show_mask(mask_tensor, num_classes=7):
    """ Display the mask using matplotlib. """
    # Convert the one-hot mask to the label mask (indices of the highest value per pixel)
    mask = mask_tensor.squeeze().cpu().numpy()  # Remove the batch dimension and convert to numpy
    label_mask = np.argmax(mask, axis=0)  # Get the class index with the highest value for each pixel

    plt.imshow(label_mask, cmap='tab20')  # Use a colormap suitable for categorical data
    plt.colorbar()  # Optionally add a color bar
    plt.axis('off')  # Hide axes
    plt.show()


class LoveDADataset(Dataset):
    def __init__(self, root_dir, split='train', transform_image=None, transform_label=None, subdir="Rural", aug1=False, aug2=False):
        self.root_dir = root_dir
        self.split = split
        self.transform_image = transform_image
        self.transform_label = transform_label
        self.aug1 = aug1
        self.aug2 = aug2
        self.image_paths = []
        self.label_paths = []

        image_dir = os.path.join(root_dir, split, subdir, 'images_png')
        label_dir = os.path.join(root_dir, split, subdir, 'masks_png')

        for f in os.listdir(image_dir):
            if f.endswith('.png'):
                self.image_paths.append(os.path.join(image_dir, f))
                self.label_paths.append(os.path.join(label_dir, f))

    def __len__(self):
        original_length = len(self.image_paths)
        if self.aug1 and self.aug2:
            return original_length * 2
        elif self.aug1 or self.aug2:
            return int(original_length * 1.5)
        else:
            return original_length


    def __getitem__(self, idx):
        original_length = len(self.image_paths)

        # Calcola l'indice originale nel dataset
        idx = idx % original_length

        aug_1 = False
        aug_2 = False
        if(idx < original_length):
            aug_1 = False
            aug_2 = False

        elif(idx >= original_length):
            if(random.random() < 0.5):
                if(self.aug1):
                    aug_1 = True
                elif(self.aug2):
                    aug_2 = True
            else:
                if (self.aug2):
                    aug_2 = True
                elif(self.aug1):
                    aug_1 = True


        # Carica l'immagine e la maschera
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = Image.open(self.label_paths[idx]).convert("L")

        image_np = np.array(image)
        label_np = np.array(label)

        # Seed per garantire coerenza tra immagine e maschera
        seed = np.random.randint(2147483647)

        # Applica augmentazione se abilitata
        if self.aug1 and aug_1:
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            transform_aug1 = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(30)
            ])
            image = Image.fromarray(image_np)
            image = transform_aug1(image)
            image_np = np.array(image)

            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            label = Image.fromarray(label_np)
            label = transform_aug1(label)
            label_np = np.array(label)

        if self.aug2 and aug_2:
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            transform_aug2 = transforms.Compose([
                #sigma=(0.1, 5) indica un intervallo di valori tra 0.1 e 5, quindi la sfocatura potrebbe variare in base a come viene applicata.
                transforms.GaussianBlur(kernel_size=(5,9), sigma=( 0.1, 5)) #valori di dimensioni (5, 9), che significa che il filtro si estende su un'area di 5x9 pixel.
            ])
            image = Image.fromarray(image_np)
            image = transform_aug2(image)
            image_np = np.array(image)

            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            label = Image.fromarray(label_np)
            label = transform_aug2(label)
            label_np = np.array(label)

        # Trasformazioni opzionali per immagine e maschera
        if self.transform_image:
            transform_image_alb = transforms.Compose([
                transforms.Resize((512, 512)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image = Image.fromarray(image_np)
            image = transform_image_alb(image)

        if self.transform_label:
            transform_label_alb = transforms.Compose([
                transforms.Resize((512, 512)),
                transforms.ToTensor()
            ])
            label = Image.fromarray(label_np)
            label = transform_label_alb(label).squeeze(0)
            label = color_to_one_hot(label, LABEL_MAP)

        return image,label

# Define transformations without augmentations
transform_image = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

transform_label = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# Set augmentation flags
AUG1 = True
AUG2 = True

BATCH_SIZE = 2 #or 32, or 64 ...
NUM_WORKERS = 2 # Number of cpu cores
PIN_MEMORY = True
PERSISTENT_WORKERS = False # Set true if you have persistent workers issues




In [7]:
!git clone https://github.com/XuJiacong/PIDNet.git
%cd content

Cloning into 'PIDNet'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 386 (delta 131), reused 125 (delta 125), pack-reused 193 (from 1)
Receiving objects: 100% (386/386), 212.80 MiB | 24.95 MiB/s, done.
Resolving deltas: 100% (184/184), done.
[Errno 2] No such file or directory: 'content'
/content


In [8]:
import sys
sys.path.append('/content/PIDNet')

In [9]:
from models.model_utils import BasicBlock, Bottleneck, segmenthead, DAPPM, PAPPM, PagFM, Bag, Light_Bag

BatchNorm2d = nn.BatchNorm2d
bn_mom = 0.1
algc = False

class PIDNet(nn.Module):

    def __init__(self, m=2, n=3, num_classes=19, planes=64, ppm_planes=96, head_planes=128, augment=True):
        super(PIDNet, self).__init__()
        self.augment = augment

        # I Branch
        self.conv1 =  nn.Sequential(
                          nn.Conv2d(3,planes,kernel_size=3, stride=2, padding=1),
                          BatchNorm2d(planes, momentum=bn_mom),
                          nn.ReLU(inplace=True),
                          nn.Conv2d(planes,planes,kernel_size=3, stride=2, padding=1),
                          BatchNorm2d(planes, momentum=bn_mom),
                          nn.ReLU(inplace=True),
                      )

        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(BasicBlock, planes, planes, m)
        self.layer2 = self._make_layer(BasicBlock, planes, planes * 2, m, stride=2)
        self.layer3 = self._make_layer(BasicBlock, planes * 2, planes * 4, n, stride=2)
        self.layer4 = self._make_layer(BasicBlock, planes * 4, planes * 8, n, stride=2)
        self.layer5 =  self._make_layer(Bottleneck, planes * 8, planes * 8, 2, stride=2)

        # P Branch
        self.compression3 = nn.Sequential(
                                          nn.Conv2d(planes * 4, planes * 2, kernel_size=1, bias=False),
                                          BatchNorm2d(planes * 2, momentum=bn_mom),
                                          )

        self.compression4 = nn.Sequential(
                                          nn.Conv2d(planes * 8, planes * 2, kernel_size=1, bias=False),
                                          BatchNorm2d(planes * 2, momentum=bn_mom),
                                          )
        self.pag3 = PagFM(planes * 2, planes)
        self.pag4 = PagFM(planes * 2, planes)

        self.layer3_ = self._make_layer(BasicBlock, planes * 2, planes * 2, m)
        self.layer4_ = self._make_layer(BasicBlock, planes * 2, planes * 2, m)
        self.layer5_ = self._make_layer(Bottleneck, planes * 2, planes * 2, 1)

        # D Branch
        if m == 2:
            self.layer3_d = self._make_single_layer(BasicBlock, planes * 2, planes)
            self.layer4_d = self._make_layer(Bottleneck, planes, planes, 1)
            self.diff3 = nn.Sequential(
                                        nn.Conv2d(planes * 4, planes, kernel_size=3, padding=1, bias=False),
                                        BatchNorm2d(planes, momentum=bn_mom),
                                        )
            self.diff4 = nn.Sequential(
                                     nn.Conv2d(planes * 8, planes * 2, kernel_size=3, padding=1, bias=False),
                                     BatchNorm2d(planes * 2, momentum=bn_mom),
                                     )
            self.spp = PAPPM(planes * 16, ppm_planes, planes * 4)
            self.dfm = Light_Bag(planes * 4, planes * 4)
        else:
            self.layer3_d = self._make_single_layer(BasicBlock, planes * 2, planes * 2)
            self.layer4_d = self._make_single_layer(BasicBlock, planes * 2, planes * 2)
            self.diff3 = nn.Sequential(
                                        nn.Conv2d(planes * 4, planes * 2, kernel_size=3, padding=1, bias=False),
                                        BatchNorm2d(planes * 2, momentum=bn_mom),
                                        )
            self.diff4 = nn.Sequential(
                                     nn.Conv2d(planes * 8, planes * 2, kernel_size=3, padding=1, bias=False),
                                     BatchNorm2d(planes * 2, momentum=bn_mom),
                                     )
            self.spp = DAPPM(planes * 16, ppm_planes, planes * 4)
            self.dfm = Bag(planes * 4, planes * 4)

        self.layer5_d = self._make_layer(Bottleneck, planes * 2, planes * 2, 1)

        # Prediction Head
        if self.augment:
            self.seghead_p = segmenthead(planes * 2, head_planes, num_classes)
            self.seghead_d = segmenthead(planes * 2, planes, 1)

        self.final_layer = segmenthead(planes * 4, head_planes, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def _make_layer(self, block, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=bn_mom),
            )

        layers = []
        layers.append(block(inplanes, planes, stride, downsample))
        inplanes = planes * block.expansion
        for i in range(1, blocks):
            if i == (blocks-1):
                layers.append(block(inplanes, planes, stride=1, no_relu=True))
            else:
                layers.append(block(inplanes, planes, stride=1, no_relu=False))

        return nn.Sequential(*layers)

    def _make_single_layer(self, block, inplanes, planes, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=bn_mom),
            )

        layer = block(inplanes, planes, stride, downsample, no_relu=True)

        return layer

    def forward(self, x):
        #print(x.shape[-1]) #512
        #print(x.shape[-2])
        width_output = x.shape[-1] // 8
        height_output = x.shape[-2] // 8

        x = self.conv1(x)
        x = self.layer1(x)
        x = self.relu(self.layer2(self.relu(x)))
        x_ = self.layer3_(x)
        x_d = self.layer3_d(x)

        x = self.relu(self.layer3(x))
        x_ = self.pag3(x_, self.compression3(x))
        x_d = x_d + F.interpolate(
                        self.diff3(x),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=algc)
        if self.augment:
            temp_p = x_

        x = self.relu(self.layer4(x))
        x_ = self.layer4_(self.relu(x_))
        x_d = self.layer4_d(self.relu(x_d))

        x_ = self.pag4(x_, self.compression4(x))
        x_d = x_d + F.interpolate(
                        self.diff4(x),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=algc)
        if self.augment:
            temp_d = x_d

        x_ = self.layer5_(self.relu(x_))
        x_d = self.layer5_d(self.relu(x_d))
        x = F.interpolate(
                        self.spp(self.layer5(x)),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=algc)

        x_ = self.final_layer(self.dfm(x_, x, x_d))

        if self.augment:
            x_extra_p = self.seghead_p(temp_p)
            x_extra_d = self.seghead_d(temp_d)
            return [x_extra_p, x_, x_extra_d]
        else:
            return x_


def get_seg_model(name, num_classes, imgnet_pretrained, model_pth):

    if 's' in name:
        model = PIDNet(m=2, n=3, num_classes=num_classes, planes=32, ppm_planes=96, head_planes=128, augment=True)
    elif 'm' in name:
        model = PIDNet(m=2, n=3, num_classes=num_classes, planes=64, ppm_planes=96, head_planes=128, augment=True)
    else:
        model = PIDNet(m=3, n=4, num_classes=num_classes, planes=64, ppm_planes=112, head_planes=256, augment=True)



    if imgnet_pretrained:

        pretrained_state = torch.load(model_pth, map_location='cpu')['state_dict']
        model_dict = model.state_dict()
        pretrained_state = {k: v for k, v in pretrained_state.items() if (k in model_dict and v.shape == model_dict[k].shape)}
        model_dict.update(pretrained_state)

        msg = 'Loaded {} parameters!'.format(len(pretrained_state))

        model.load_state_dict(model_dict, strict = False)

    else:
        # print("vamos")

        pretrained_dict = torch.load(model_pth, map_location='cpu')
        if 'state_dict' in pretrained_dict:
            pretrained_dict = pretrained_dict['state_dict']
        model_dict = model.state_dict()
        pretrained_dict = {k[6:]: v for k, v in pretrained_dict.items() if (k[6:] in model_dict and v.shape == model_dict[k[6:]].shape)}
        msg = 'Loaded {} parameters!'.format(len(pretrained_dict))
        logging.info('Attention!!!')
        logging.info(msg)
        logging.info('Over!!!')
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict, strict = False)

    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_seg_model(name='pidnet_s', num_classes=7, imgnet_pretrained=True, model_pth='PIDNet_S_ImageNet.pth.tar').to(device)




<ipython-input-9-34d80a4c5901>:189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state = torch.load(model_pth, map_location='cpu')['state_dict']


In [10]:
#device="cpu"
print(device)


cuda


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
import random
from sklearn.metrics import mean_squared_error

import random
import torch
import torch.nn.functional as F

num_classes=7
# Define mIoU metric
jaccard = JaccardIndex(task="multiclass", num_classes=7).to('cuda')  # intersection over union. Directly measures the overlap between predicted segmentation and ground truth.

# DACS implementation (adapting from the pseudocode)
class DACSModel(nn.Module):
    def __init__(self, num_classes):
        super(DACSModel, self).__init__()
        self.model = get_seg_model(name='pidnet_s', num_classes=7, imgnet_pretrained=True, model_pth='PIDNet_S_ImageNet.pth.tar').to(device)

    def forward(self, x):
        return self.model(x)

def dacs_loss(pred_source, label_source, pred_mixed, label_mixed, lambda_param): #from the paper.
    # Cross-entropy loss for source domain and mixed domain
    criterion = nn.CrossEntropyLoss()

    #print("pred_source.shape: ", pred_source.shape)
    #print("label_source.shape: ", label_source.shape)
    #print("pred_mixed.shape: ", pred_mixed.shape)
    #print("label_mixed.shape: ", label_mixed.shape)

    # Calculate loss for both source and mixed data
    loss_source = criterion(pred_source, label_source)
    #print("loss_source: ", loss_source)
    loss_mixed = criterion(pred_mixed, label_mixed)
    #print("loss_mixed: ", loss_mixed)

    return loss_source + lambda_param * loss_mixed


def classmix(source_images, source_labels, target_images, target_labels, num_classes=7):
    """
    Perform classmix image augmentation, mixing source domain image and target domain image pixels based on class masks.
    """
    batch_size, _, height, width = source_images.size()
    #print("sourceLabels",source_labels)

    # Ensure source_labels is an integer type
    source_labels = source_labels.long()

    # Generate a random binary mask for each class
    class_masks = torch.zeros((batch_size, height, width), device=source_images.device, dtype=torch.bool)
    #print(class_masks)
    #print(class_masks.shape) #torch.Size([2, 512, 512])
    #print(source_labels.shape) #torch.Size([2, 7, 512, 512])
    #source_labels = torch.argmax(source_labels, dim=1)  # Convert to class indices (batch_size, height, width)

    for b in range(batch_size):
        # Randomly select half of the classes
        selected_classes = torch.randperm(num_classes)[:num_classes // 2]
        for cls in selected_classes:
            class_masks[b] = class_masks[b] | (source_labels[b] == cls)

    # Expand the mask to match the channel dimension for images
    class_masks_expanded = class_masks.unsqueeze(1).expand(-1, source_images.size(1), -1, -1)

    # Mix images using the masks
    mixed_images = torch.where(class_masks_expanded, source_images, target_images)

    # Mix labels using the same masks
    mixed_labels = torch.where(class_masks, source_labels, target_labels)


    return mixed_images, mixed_labels


def calculate_miou(predictions, labels, num_classes=7):
    """
    Compute Mean Intersection over Union (mIoU) for a given batch of mixed images.
    Args:
    - predictions (torch.Tensor): Predicted labels for each pixel in the batch (B, H, W)
    - labels (torch.Tensor): Ground truth labels for each pixel in the batch (B, H, W)
    - num_classes (int): The total number of classes for segmentation
    Returns:
    - mean_iou (float): The mean Intersection over Union (mIoU) for the batch
    """
    iou_per_class = []

    # Loop over each class
    for cls in range(num_classes):
        # Predicted pixels belonging to the class
        pred_class = (predictions == cls)
        # Ground truth pixels belonging to the class
        true_class = (labels == cls)

        # Intersection (True Positive)
        intersection = torch.sum(pred_class & true_class).float()
        # Union (True Positive + False Positive + False Negative)
        union = torch.sum(pred_class | true_class).float()

        # IoU for the class
        if union != 0:
            iou = intersection / union
        else:
            iou = torch.tensor(0.0)

        iou_per_class.append(iou)

    # Calculate the mean IoU for the batch
    mean_iou = torch.mean(torch.stack(iou_per_class))
    return mean_iou

def validate(model, target_loader, num_classes=7): #we give it the model trained on mixed images and mixed labels
    model.eval()  # Set model to evaluation mode
    running_miou = 0.0
    total_samples = 0

    with torch.no_grad():  # No gradient computation needed for validation
        for target_data in target_loader:
            target_images, target_labels = target_data
            target_images, target_labels = target_images.to(device), target_labels.to(device)

            # 1. Forward pass for target domain images
            target_preds = model(target_images)
            main_output_target = target_preds[1]
            main_output_target = F.interpolate(main_output_target, size=(512, 512), mode='bilinear', align_corners=True)

            # 2. Calculate mIoU for target domain
            miou_batch = calculate_miou(main_output_target, target_labels, num_classes)
            running_miou += miou_batch.item()
            total_samples += 1

    mean_miou = running_miou / total_samples
    print(f"Validation mIoU: {mean_miou:.4f}")
    return mean_miou

def calculate_lambda(predictions, threshold=0.7):
    """
    Calculate the adaptive lambda based on the proportion of confident pixels.

    Args:
    - predictions (torch.Tensor): Model predictions (logits or probabilities) [batch_size, num_classes, H, W].
    - threshold (float): Confidence threshold for determining confident pixels.

    Returns:
    - lambda_value (float): Adaptive lambda for the current batch.
    """
    # Convert logits to probabilities (if necessary)
    probs = F.softmax(predictions, dim=1)  # [batch_size, num_classes, H, W]

    # Get maximum probabilities for each pixel
    max_probs, _ = probs.max(dim=1)  # [batch_size, H, W]

    # Create a mask of confident pixels
    confident_mask = max_probs > threshold  # [batch_size, H, W]

    # Calculate the proportion of confident pixels for each image in the batch
    confident_proportions = confident_mask.float().mean(dim=(1, 2))  # [batch_size]

    # Average over the batch to get the lambda value
    lambda_value = confident_proportions.mean().item()

    return lambda_value

# DACS training loop
def train_dacs_and_validate(model, source_loader, target_loader, num_epochs=20, lambda_param=0.7, lr=1e-4, confidence_threshold=0.7):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)  # Move model to device (CUDA or CPU)
    model.train()
    best_miou = 0.0

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_miou = 0.0
        total_samples = 0

        # Iterate through both source and target datasets for training
        train_progress_bar = tqdm(zip(source_loader, target_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch", total=min(len(source_loader), len(target_loader)))
        for (source_data, target_data) in train_progress_bar:

            source_images, source_labels = source_data
            target_images, _ = target_data  # Target labels are not available, will use pseudo-labels

            # Move to device
            source_images, source_labels = source_images.to(device), source_labels.to(device)
            target_images = target_images.to(device)

            # 1. Forward pass through the model for the source and target domains
            source_preds = model(source_images)
            main_output_source= source_preds[1]
            #print("main_output_source shape : ", main_output_source.shape)
            main_output_source = F.interpolate(main_output_source, size=(512,512), mode='bilinear', align_corners=True)
            #print("main_output_source shape : ", main_output_source.shape) #torch.Size([2, 7, 512, 512])
            #print("logits", main_output_source[0, :, 0, 0]) #example: tensor([-3.4636, -3.5338,  2.0996, -0.4825, -0.7998,  1.7863,  2.0860], device='cuda:0', grad_fn=<SelectBackward0>)


            # 2. Generate pseudo-label for target domain
            target_preds = model(target_images)
            main_output_target = target_preds[1]
            #print("main_output_target shape : ", main_output_target.shape)
            main_output_target = F.interpolate(main_output_target, size=(512,512), mode='bilinear', align_corners=True)
            #print("main_output_target shape : ", main_output_target.shape) #torch.Size([2, 7, 512, 512])

            #The goal of pseudo-labeling is to assign a single class label to each pixel based on the highest probability (or logit). The argmax function finds the index of the maximum value along the specified dimension (num_classes). The result tensor is (batch_size, height, width)
            pseudo_labels = torch.argmax(main_output_target, dim=1).to(device)
            #print("pseudo_labels", pseudo_labels.shape) #torch.Size([2, 512, 512])
            #print("source labels aaaaaa:", source_labels) #here each class label is represented as a one-hot vector for each pixel across the image. we want convert the one-hot encoded source_labels into class indices, so each pixel gets assigned the class marked as 1 in the one-hot vector.
            #If a pixel in source_labels has a one-hot encoded value like [1, 0, 0, 0], this means that the pixel belongs to class 0 (the first class). ->
            source_labels = torch.argmax(source_labels, dim=1).to(device)
            #print("source_labels", source_labels.shape) # torch.Size([2, 512, 512])

            # 3. Calculate adaptive lambda
            #lambda_param = calculate_lambda(main_output_target, threshold=confidence_threshold)
            #train_progress_bar.set_postfix({'lambda': lambda_param})

            # 4. MIX source and target images and labels (DACS augmentation step)
            mixed_images, mixed_labels = classmix(source_images, source_labels, target_images, pseudo_labels)
            #print("mixed_labels ", mixed_labels.shape)
            mixed_labels=mixed_labels.to(device)
            #print("mixed_labels", mixed_labels)

            # 5. Forward pass for mixed data
            mixed_preds = model(mixed_images)
            main_output_mixed = mixed_preds[1]
            #print("main_output_mixed shape : ", main_output_mixed.shape)
            main_output_mixed = F.interpolate(main_output_mixed, size=(512,512), mode='bilinear', align_corners=True)
            #print("main_output_mixed shape : ", main_output_mixed.shape) #torch.Size([2, 7, 512, 512])
            #print("main_output_mixed", main_output_mixed)


            # Calculate the loss
            loss = dacs_loss(main_output_source, source_labels, main_output_mixed, mixed_labels, lambda_param)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print loss and mIoU after each epoch
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {loss.item():.4f}")

        # Clear GPU memory
        torch.cuda.empty_cache()

        # Validation loop
        model.eval()
        val_miou = 0.0
        val_class_iou = torch.zeros(len(LABEL_MAP)).to('cuda')  # Store IoU per class
        val_class_counts = torch.zeros(len(LABEL_MAP)).to('cuda')  # To track number of pixels for each class

        with torch.no_grad():
            for images, masks in target_loader:
                images, masks = images.to('cuda'), masks.to('cuda')

                # Forward pass
                outputs = model(images)  # [batch_size, num_classes, 512, 512]
                main_output = outputs[1]
                main_output = F.interpolate(main_output, size=(512,512), mode='bilinear', align_corners=True)
                preds = torch.argmax(main_output, dim=1)  # [batch_size, 512, 512]

                # Convert masks to class indices format if one-hot encoded
                if masks.ndim == 4:  # [batch_size, num_classes, height, width]
                    masks = masks.argmax(dim=1)  # [batch_size, height, width]

                # Calculate per-class IoU
                for c, class_name in LABEL_MAP.items():  # Iterate over LABEL_MAP classes
                    true_class = (masks == class_name)
                    pred_class = (preds == class_name)

                    intersection = torch.sum(true_class & pred_class).float()
                    union = torch.sum(true_class | pred_class).float()

                    if union != 0:
                        val_class_iou[class_name] += intersection / union
                    val_class_counts[class_name] += 1

                # Calculate overall mIoU for this batch
                val_miou += jaccard(preds, masks)

        # Average metrics
        val_miou /= len(target_loader)

        # Calculate average IoU for each class
        avg_class_iou = val_class_iou / val_class_counts

        # Print validation metrics at the end of each epoch
        print(f"Epoch [{epoch+1}/{num_epochs}] - Validation mIoU: {val_miou:.4f}")
        print(f"Epoch [{epoch+1}/{num_epochs}] - Per-Class IoU: {dict(zip(LABEL_MAP.keys(), avg_class_iou.tolist()))}")

        # Save the best model
        if val_miou > best_miou:
            best_miou = val_miou
            torch.save(model.state_dict(), 'best_pidnet_model.pth')



# Loading datasets
source_dataset = LoveDADataset(root_dir='Train/', split='Train', transform_image=transform_image, transform_label=transform_label, subdir='Urban', aug1=AUG1, aug2=AUG2)
target_dataset = LoveDADataset(root_dir='Val/', split='Val', transform_image=transform_image, transform_label=transform_label, subdir='Rural')

source_loader = DataLoader(source_dataset, batch_size=2, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, persistent_workers=PERSISTENT_WORKERS)
target_loader = DataLoader(target_dataset, batch_size=2, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)


# Define the size of the subset (e.g., 10% of the dataset)
# subset_size = 0.2  # 10% of the dataset
# source_indices = list(range(len(source_dataset)))
# target_indices = list(range(len(target_dataset)))

# # Select a subset of indices
# source_subset_indices = random.sample(source_indices, int(len(source_indices) * subset_size))
# target_subset_indices = random.sample(target_indices, int(len(target_indices) * subset_size))

# # Create subset datasets
# source_subset = Subset(source_dataset, source_subset_indices)
# target_subset = Subset(target_dataset, target_subset_indices)

# # Create data loaders for the subset datasets
# source_loader = DataLoader(source_subset, batch_size=2, shuffle=True)
# target_loader = DataLoader(target_subset, batch_size=2, shuffle=True)


# Initialize, train the model and validate
model = DACSModel(num_classes=7).to(device)  # Assuming 7 classes
train_dacs_and_validate(model, source_loader, target_loader, num_epochs=20)




<ipython-input-9-34d80a4c5901>:189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state = torch.load(model_pth, map_location='cpu')['state_dict']
Epoch 1/20: 100%

Epoch [1/20] - Loss: 1.8365


Epoch [1/20] - Validation mIoU: 0.1039
Epoch [1/20] - Per-Class IoU: {'Background': 0.37344419956207275, 'Building': 0.1255132555961609, 'Road': 0.10872494429349899, 'Water': 0.06966084241867065, 'Barren': 0.016518406569957733, 'Forest': 0.03077808953821659, 'Agricultural': 0.002549270633608103}


Epoch 2/20: 100%|██████████| 496/496 [03:12<00:00,  2.58batch/s]

Epoch [2/20] - Loss: 1.1617


...

Epoch [19/20] - Loss: 0.7039


Epoch [19/20] - Validation mIoU: 0.1404
Epoch [19/20] - Per-Class IoU: {'Background': 0.10732676088809967, 'Building': 0.14357803761959076, 'Road': 0.1574588567018509, 'Water': 0.19725535809993744, 'Barren': 0.02062210813164711, 'Forest': 0.02178467996418476, 'Agricultural': 0.3286987245082855}


Epoch 20/20: 100%|██████████| 496/496 [03:13<00:00,  2.57batch/s]

Epoch [20/20] - Loss: 0.7704


Epoch [20/20] - Validation mIoU: 0.1372
Epoch [20/20] - Per-Class IoU: {'Background': 0.1027241200208664, 'Building': 0.14174510538578033, 'Road': 0.14628703892230988, 'Water': 0.20249570906162262, 'Barren': 0.013422747142612934, 'Forest': 0.027786964550614357, 'Agricultural': 0.3181988596916199}


In [12]:
# unsqueeze function in PyTorch is used to add an extra dimension to a tensor.
# channels: 3 color channels (RGB)

DACS method:
1. ClassMix strategy:
Half of the classes from a source domain image are selected, and the corresponding pixels are cut out and pasted onto an image from the target domain.
The same mixing procedure is applied to the labels (pseudo-labels for the target domain and ground-truth labels for the source domain).
2. Pseudo-labels:
The target domain image is passed through the model before mixing to produce pseudo-labels. These labels are then mixed similarly to how the images are mixed.
3. Cross-domain Mixing:
By combining the two domains at the pixel level, this method attempts to make it harder for the model to learn to discern between the domains. This can mitigate the issue of class distribution differences between source and target domains.

Key Steps for Implementation:
1. Image Mixing:
Select a portion of the source domain image and mix it with the target domain image.
2. Label Mixing:
Use the ground-truth label from the source domain and pseudo-labels from the target domain and mix them similarly to the images.
3. Model Training:
Once the mixed images and labels are prepared, the model can be trained using the usual supervised learning approach, where predictions are made and loss is computed using cross-entropy.

1. Training Phase:

Input: Source images and source labels.

Output: Predicted masks and loss between predictions and source labels.

Augment with target data (via classmix).

Backpropagation and optimization.

2. Validation Phase:

Input: Target images (no labels for target during training).

Output: Predicted masks for the target images.

Compute metrics such as mIoU between predicted masks and true target domain masks.